<a href="https://colab.research.google.com/github/Valiev-Koyiljon/Face-mask-detection-Pytorch/blob/main/RealTimeFaceMaskDetecter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from PIL import Image, ImageDraw
import numpy as np
import cv2
from mtcnn import MTCNN
import torch
import torch.nn as nn
from torchvision import transforms, models



# Loading ready model
state_dict = torch.load('Final_FaceMask_detection_model.pt', map_location=torch.device('cpu'))


class FaceMaskModel(nn.Module):
    def __init__(self):
        super(FaceMaskModel, self).__init__()
        self.model = models.resnet50(pretrained=True)
        num_features = self.model.fc.in_features
        self.model.fc = nn.Linear(num_features, 2)

    def forward(self, x):
        return self.model(x)




# Face mask detection model
model = FaceMaskModel()
model.load_state_dict(state_dict)
model.eval()


# Transformation for input image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# labels (with mask and without mask)
labels = ['Without Mask', 'With Mask']



detector = MTCNN()



# Initialize the video capture
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Failed to open the camera")
    exit()
    
    

# Loop over frames from the video stream
while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame")
        break

        
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    
    # face detection using MTCNN
    boxes = detector.detect_faces(rgb_frame)

    
    
    # detected faces
    for box in boxes:
        x, y, width, height = box['box']

    
    
    
    
     #face region from the frame
        face = frame[y:y+height, x:x+width]

        
        
        #  face to PIL Image format
        pil_face = Image.fromarray(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))

        
        transformed_face = transform(pil_face)

        
        
        # a batch dimension to the face image
        transformed_face = transformed_face.unsqueeze(0)

        
        with torch.no_grad():
            outputs = model(transformed_face)
            _, predicted = torch.max(outputs, 1)

        
        
        label = labels[predicted.item()]

        # bounding box and lebel
        cv2.rectangle(frame, (x, y), (x+width, y+height), (0, 255, 0), 2)
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        
    cv2.imshow('Face Mask Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
        

cap.release()
cv2.destroyAllWindows()


C:\Users\he\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\he\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


1/1 [==============================] - 0s 63ms/step


1/1 [==============================] - 0s 57ms/step


1/1 [==============================] - 0s 42ms/step


1/1 [==============================] - 0s 47ms/step
